# He_line_calc: a notebook for reducing He line data

## Introduction

This notebook reduces data produced by a Pfieffer PrismaPlus220 in the Helium Analysis Laboratory (HAL) at the University of Illinois. The HAL uses h6t and Pychron software for data reporting and measures masses 1-5 on line blanks, hot blanks, line gas standards, and samples. We use an isotope dilution approach with a $^3$He spike and a $^4$He reference gas of a known volume. As such, 4/3 gas ratios are measured with corrections for H, D, and HD. This notebook reads in the raw data files from either h6t or Pychron and reports out He amounts in terms of pmol.

Instructions for the use of this notebook are provided before each cell of code and should be followed step-by-step. Only one notebook is needed for a complete set of analyses that may encompass multiple days (referred throughout in this notebook as a "session"). Some cells will be run at the very beginning of data collection, whereas other cells will be run repeatedly as new data is collected. __Pay attention as to which cells need to be run once, and which will be repeatedly run throughout the data collection process!__ If you are unfamiliar with Jupyter Notebooks, the "run" button is found at the top of the notebook, and each cell can be run by highlighting it with the mouse and then clicking the "run" button.

### Step-by-step instructions

First, check to see if there is any output left over for any of the cells from the last time the notebook was run. If there is, then go to Kernel -> Restart & Clear Output before proceeding.

This first cell imports some useful packages, sets some constants, and intializes several key running means that will be used throughout the notebook. __This cell should be run only once! If it is rerun by accident before you have finished a sample run, you will need to reprocess all of your files in the project folder in chronologic order. So be careful not to rerun this first cell!__

In [ ]:
import pandas as pd
import numpy as np
import math
import json
import csv
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
%matplotlib inline
from datetime import datetime

#constants
ideal_gas_moles = 22.414 #liter/mol
initial_tank_4He = 6.8630
initial_tank_4He_1s = 0.0489
tank_depletion = 0.999981

#initialize mean lists, lists are needed here to calculate standard deviations
#initialize reference list of lists and sample dictionary, ref lists tie running means to specific samples
hb_list = []
lb_list = []
std_list = []
std_num_list = []
hb_ref_list = []
lb_ref_list = []
std_ref_list = []
sample_dict = {}

#initialize sample times and reference numbers used to tie samples to relevant running means 
last_hb_time = 0
last_lb_time = 0
last_std_time = 0
std_ref_num = -1
lb_ref_num = -1
hb_ref_num = -1

#this data frame is for display purposes only
aliquot_frame = pd.DataFrame(columns = ['4He/3He intercept','4He/3He err', 'notes'], index = [])

This next cell of code reads each individual run file produced by h6t. You will need to __run this cell everytime you collect new data from the PrismaPlus__. In the cell below, enter __1) the name of the aliquot in the aliquot variable, 2) the project name in the project variable, and 3) the QST number in the QST variable. You will need to type within the ''.__ The file extension is added to the aliquot name in the next line of code. The aliquot name should be the name of the sample, line blank, hot blank, or line standard. Line blanks have the style: 'lbXX_mmddyyyy' where XX = the number for the line blank on that day (01, 02, 03 ... 10, 11, etc.), mm = the month, dd = the day, and yyyy = the year. Hot blanks have the style: 'hbXX_mmddyyyy'. Line standards have the style: 'stdXXXX' where XXXX is the shot number from the $^4$He pipette as recorded in the notebook and listed in the 4HeTank actuation number (the shot number or QRT is double the actuation number). Re-extracts are appended to the end of the sample name with the convention: 'sample_reX' where 'X' is the re-extract number (1-4). __You must follow these naming conventions or the rest of the code does not work!__

In [ ]:
#fill in information here between the '' and after QST = 
aliquot = 'std1012'
QST = 6726

project = 'h6T_jan2023_test'

#finding the file
file_path = os.path.join('C:\\','Users','lab-admin','Documents','GitHub','HAL_data','projects_2023',project)
os.chdir(file_path)
file_name_list = os.listdir('.')
actuation = str(QST*2)

for file in file_name_list:
    if actuation in file:
        file_name = file
        
#creating prisma data list
Prisma_data_list = []
with open(file_name, mode='r') as in_file:
    csv_reader_He = csv.reader(in_file)
    
    #throw away first line of headers
    next(csv_reader_He)
    
    #read in the data lines to a list of lists until the end
    for line in csv_reader_He:
        #throw away empty rows in between lines
        if any(line):
            Prisma_data = [float(i) for i in line[:7]]
            Prisma_data_list.append(Prisma_data)


#convert data_list to an array for easier indexing
Prisma_data_array=np.array(Prisma_data_list)

aliquot_time = Prisma_data_array[-1,1]

#create time list (x-values) and corrected 4He/3He list (y-values)
t_list = [(Prisma_data_array[i,1]-Prisma_data_array[0,1]) for i in range(len(Prisma_data_array))]
He_ratio_list = [(Prisma_data_array[i,5]-Prisma_data_array[i,6])/(Prisma_data_array[i,4]-Prisma_data_array[i,6]-0.005*Prisma_data_array[i,2]) for i in range(len(Prisma_data_array))]

#do some math to find the intercept and mean of the corrected 4He/3He
sum_t_y = 0
sum_t2 = 0
sum_slope_err = 0

for i in range(len(t_list)):
    sum_t_y = sum_t_y + t_list[i]*He_ratio_list[i]
    sum_t2 = sum_t2 + t_list[i]**2

slope = (len(t_list)*sum_t_y - sum(t_list)*sum(He_ratio_list))/(len(t_list)*sum_t2 - sum(t_list)**2)
intercept = (sum(He_ratio_list) - slope*sum(t_list))/len(t_list)

for i in range(len(t_list)):
    sum_slope_err = sum_slope_err + (He_ratio_list[i] - intercept - slope*t_list[i])**2

del_slope = math.sqrt(sum_slope_err/(len(t_list) - 2)) * math.sqrt(len(t_list)/(len(t_list)*sum_t2 - sum(t_list)**2))
del_intercept = math.sqrt(sum_slope_err/(len(t_list) - 2)) * math.sqrt(sum_t2/(len(t_list)*sum_t2 - sum(t_list)**2))

mean_4He_3He = np.mean(He_ratio_list)
stdev_4He_3He = np.std(He_ratio_list)

print('The int and err for ', aliquot,'is ',intercept,' +/- ',del_intercept)
print('and the mean and std dev is ',mean_4He_3He,' +/- ',stdev_4He_3He)

#show graph of masses through time
seconds_list = []
for i in range(len(Prisma_data_array[:,1])):
    time_since_epoch = datetime.fromtimestamp(Prisma_data_array[i,1])
    seconds_list.append(mdate.date2num(time_since_epoch))
date_fmt = '%m-%d-%y %H:%M:%S'
date_formatter = mdate.DateFormatter(date_fmt)

plt.figure(figsize = (14,12))
plt.ylabel('intensity')
plt.yscale('log')
plt.xlabel('time')
plt.plot(seconds_list, Prisma_data_array[:,2], label = 'H')
plt.plot(seconds_list, Prisma_data_array[:,3], label = 'H2 + D')
plt.plot(seconds_list, Prisma_data_array[:,4], label = 'HD + 3He')
plt.plot(seconds_list, Prisma_data_array[:,5], label = '4He')
plt.plot(seconds_list, Prisma_data_array[:,6], label = 'background')
plt.gca().xaxis.set_major_formatter(date_formatter)
plt.legend()
plt.show()

Now you're ready to add these data to the running line summary list for your run. __If you do not want the data to be added to the running list do not run the next sequence of cells.__ There are various reasons why you might not want to add this particular aliquot to the running total. For example, typically the first line blank of the day comes in with a slightly high 4He/3He ratio due to static conditions in the line (valves closed for a prolonged period of time) and we discard it.

If you do want to add these data to the line summary list, click run in the next cell below to proceed. __Add any important sample notes to the sample data frame by filling in the '' next to the notes variable.__ The assumption here is that all data added to a given line summary list will be in chronologic order.

In [ ]:
#add sample/run to sample data frame for display purposes at the end of cell
#enter in any notes you have in the notes variable below

notes = ''
row_data = pd.DataFrame({'4He/3He intercept':intercept,'4He/3He err':del_intercept,'notes':notes, 'file':file_name },columns = ['4He/3He intercept','4He/3He err', 'notes', 'file'], index = [aliquot])
aliquot_frame = pd.concat([aliquot_frame, row_data])

#selection statements to determine type of aliquot and what to do with it
#blanks and standards have additional selection statements in order to reset running means if long periods of time
#(8+ hours) have elapsed between measurements
#re-extracts are dealt with by adding their totals to the correct sample, NOTE: THIS ASSUMES THAT RE-EXTRACTS AND 
#THEIR SAMPLES ARE CO-LOCATED IN THE SAME LINE SUMMARY NOTEBOOK!

if aliquot.startswith('hb') or aliquot.startswith('HB'):
    if (aliquot_time - last_hb_time)> 8*60*60:
        #reset hb_list if longer than 8 hours between hot blanks or first hot blank
        hb_list = []
        hb_list.append(intercept)
        hb_mean = np.mean(hb_list)
        hb_err = del_intercept
        hb_ref_list.append([hb_mean, hb_err])
        last_hb_time = aliquot_time
        hb_ref_num = hb_ref_num + 1
    else:
        hb_list.append(intercept)
        hb_mean = np.mean(hb_list)
        hb_err = np.std(hb_list)
        hb_ref_list[hb_ref_num - 1] = [hb_mean, hb_err]
        last_hb_time = aliquot_time
elif aliquot.startswith('lb') or aliquot.startswith('LB'):
    if (aliquot_time - last_lb_time)> 8*60*60:
        #reset lb_list if longer than 8 hours between line blanks or first line blank
        lb_list = []
        lb_list.append(intercept)
        lb_mean = np.mean(lb_list)
        lb_err = del_intercept
        lb_ref_list.append([lb_mean, lb_err])
        last_lb_time = aliquot_time
        lb_ref_num = lb_ref_num + 1
    else:
        lb_list.append(intercept)
        lb_mean = np.mean(lb_list)
        lb_err = np.std(lb_list)
        lb_ref_list[lb_ref_num - 1] = [lb_mean, lb_err]
        last_lb_time = aliquot_time
elif aliquot.startswith('std') or aliquot.startswith('STD'):
    if (aliquot_time - last_std_time)> 8*60*60:
        #reset std_list and std_num_ list if longer than 8 hours between standards or first standard
        std_list = []
        std_list.append(intercept)
        std_num_list = []
        std_num_list.append(float(aliquot[3:]))
        std_mean = np.mean(std_list)
        std_err = del_intercept
        std_num = np.mean(std_num_list)
        std_ref_list.append([std_mean, std_err, std_num])
        last_std_time = aliquot_time
        std_ref_num = std_ref_num + 1
    else:
        std_list.append(intercept)
        std_num_list.append(float(aliquot[3:]))
        std_mean = np.mean(std_list)
        std_err = np.std(std_list)
        std_num = np.mean(std_num_list)
        std_ref_list[std_ref_num - 1] = [std_mean, std_err, std_num]
        last_std_time = aliquot_time
elif aliquot[-4:-1] == '_re':
    reextract_sample = aliquot.split('_re')
    update_list = sample_dict[reextract_sample[0]]
    update_list[0] = update_list[0] + intercept
    update_list[1] = update_list[1] + del_intercept
    update_list[2] = update_list[2] + 1
    sample_dict[reextract_sample[0]] = update_list
else:
    sample_dict[aliquot] = [intercept, del_intercept, 0, aliquot_time, lb_ref_num, hb_ref_num, std_ref_num, notes]

aliquot_frame

In [ ]:
sample_dict
hb_ref_list

This last step organizes and reduces your data for export to a json file that we use to further reduce the data once U, Th, and Sm information has been collected. Here, the notebook reports out blank corrected, 4He volumes (in ncc) and amounts (in nmols) for each sample aliquot. We use the hot blank as the blank correction as it most closely resembles the valve procedures for running samples. The line blank could be substituted here. __You will only need to run this once at the end to export the data.__

In [ ]:
#json dictionary that gets reported out
json_out_dict = {}

#for loop that steps through elements of sample_dict, converts to volume and amount, and adds to the json_out_dict
for index in sample_dict:
    
    std_ncc = initial_tank_4He * tank_depletion**std_ref_list[sample_dict[index][6]][2]
    std_ncc_1s = initial_tank_4He_1s * tank_depletion**std_ref_list[sample_dict[index][6]][2]
    sample_vol = ((sample_dict[index][0] - (sample_dict[index][2] + 1)*hb_ref_list[sample_dict[index][5]][0])/
                    (std_ref_list[sample_dict[index][6]][0]-hb_ref_list[sample_dict[index][5]][0])) * std_ncc
    sample_mol = sample_vol * 1e-12/ideal_gas_moles
    
    #propagate errors
    d_ncc4He_d_samp = std_ncc/(sample_dict[index][0] - hb_ref_list[sample_dict[index][5]][0])
    d_ncc4He_d_stdncc = (sample_dict[index][0] - hb_ref_list[sample_dict[index][5]][0])/std_ncc
    d_ncc4He_d_std = -(sample_dict[index][0] - hb_ref_list[sample_dict[index][5]][0])*std_ncc/(std_ref_list[sample_dict[index][6]][0] - hb_ref_list[sample_dict[index][5]][0])**2
    sample_vol_err = math.sqrt(sample_dict[index][1]**2 * d_ncc4He_d_samp**2 + std_ncc_1s**2 * d_ncc4He_d_stdncc**2
                               + std_ref_list[sample_dict[index][6]][1]**2 * d_ncc4He_d_std**2)
    sample_mol_err = sample_vol * 1e-12/ideal_gas_moles
    
    #add to the json_dict
    json_out_dict[index] = [sample_vol, sample_vol_err, sample_mol, sample_mol_err, sample_dict[index][7]]
    
#send the dict to a json file

now = datetime.now()
json_file = 'He_line_summary_' + now.strftime('%m%d%Y') + '.json'
csv_file = 'He_line_summary_' + now.strftime('%m%d%Y') + '.csv'

with open(json_file, 'w') as out_file:
    json.dump(json_out_dict, out_file)

#save the aliquot_frame to a csv file
aliquot_frame.to_csv(csv_file)    

Once you have successfully exported your data and confirmed that the file is safe and sound, go to Kernel -> Restart & Clear Output to reset the notebook for the next use. Save and exit the notebook.